### iGED : Global Systems Dynamics Initiative

El objetivo principal de este documento es analizar las métricas globales en las siguientes redes capitales
- Ciudad Autónoma de Buenos Aires (CABA)
- Ciudad de México (CDMX)
- Santiago de Chile (SCL)
- Montevideo (MTV)
- Madrid (MAD)
- Sao Paulo (SAO)

y obtener un tidy DataFrame, para continuar analizando los datos obtenidos, por ejemplo obteniendo correlaciones entre parejas de métricas.  


In [2]:
#-------------------------------------------------------
# Importar paquetes a utilizar
#-------------------------------------------------------
import pandas as pd
import numpy as np 
import networkx as nx
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

#-------------------------------------------------------
# Archivo CSV describiendo los nodos de cada red
#-------------------------------------------------------
ags_nd=pd.read_csv('Gephi_stats/Gephi AGS Stats.csv')             
caba_nd=pd.read_csv('Gephi_stats/Gephi CABA Stats.csv')           #capital
cdmx_nd=pd.read_csv('Gephi_stats/Gephi CDMX Stats.csv')           #capital
gdl_nd=pd.read_csv('Gephi_stats/Gephi GDL Stats.csv') 
hgo_nd=pd.read_csv('Gephi_stats/Gephi Hidalgo Stats.csv')
mad_nd=pd.read_csv('Gephi_stats/Gephi Madrid Stats.csv')          #capital
mtv_nd=pd.read_csv('Gephi_stats/Gephi Montevideo Stats.csv')      #capital
oax_nd=pd.read_csv('Gephi_stats/Gephi Oaxaca Stats.csv')
sao_nd=pd.read_csv('Gephi_stats/Gephi Sao Paulo Stats.csv')       #capital
scl_nd=pd.read_csv('Gephi_stats/Gephi SCL Stats.csv')             #capital


#-------------------------------------------------------
# Archivo CSV describiendo las aristas de cada red
#-------------------------------------------------------
ags_ed=pd.read_csv('Gephi_edges/Gephi AGS Edges.csv')
caba_ed=pd.read_csv('Gephi_edges/Gephi CABA Edges.csv')           #capital
cdmx_ed=pd.read_csv('Gephi_edges/Gephi CDMX Edges.csv')           #capital
gdl_ed=pd.read_csv('Gephi_edges/Gephi GDL Edges.csv')
hgo_ed=pd.read_csv('Gephi_edges/Gephi Hidalgo Edges.csv')
mad_ed=pd.read_csv('Gephi_edges/Gephi Madrid Edges.csv')          #capital
mtv_ed=pd.read_csv('Gephi_edges/Gephi Montevideo Edges.csv')      #capital
oax_ed=pd.read_csv('Gephi_edges/Gephi Oaxaca Edges.csv')
sao_ed=pd.read_csv('Gephi_edges/Gephi Sao Paulo Edges.csv')       #capital
scl_ed=pd.read_csv('Gephi_edges/Gephi SCL Edges.csv')             #capital

FileNotFoundError: [Errno 2] File Gephi_stats/Gephi AGS Stats.csv does not exist: 'Gephi_stats/Gephi AGS Stats.csv'

In [ ]:
def armar_grafo(nodes,edges,rol_str,weight_str):
    '''
    Función con la cual, a partir de una lista de nodos y conexiones, forma un grafo con NetworkX
    
    In:
    - nodes       lista de nodos
    - edges       lista de aristas
    - rol_str     un nombre para el parámetro que describe el rol de un actor
    - weight_str  un nombre para el parámetro que describe el peso de las aristas
    
    Out
    Un objeto NetworkX llamado G.
    '''
    
    #crea un grafo dirigido a partirde la lista edges
    G=nx.from_pandas_edgelist(edges,'Source','Target',edge_attr=["Weight"],create_using=nx.DiGraph())
    
    #rol es un diccionario que manda cada id de un nodo a el atributo correspondiente a rol
    rol = {nid: nodes[nodes['Id']==nid][rol_str].values[0] for nid in nodes['Id']}
    nx.set_node_attributes(G,rol,'rol')
    
    #weight es un diccionario que manda cada id de un nodo a el atributo correspondiente al peso de nodo
    weight = {nid: nodes[nodes['Id']==nid][weight_str].values[0] for nid in nodes['Id']}
    nx.set_node_attributes(G,weight,'weight')
    
    return G

In [ ]:
#--------------------------------------------------------------
#Armar grafos a partir de cada uno de los CSV que descargamos
#--------------------------------------------------------------

ags_G=armar_grafo(ags_nd,ags_ed,'role','weight')
caba_G=armar_grafo(caba_nd,caba_ed,'type','weight')
cdmx_G=armar_grafo(cdmx_nd,cdmx_ed,'rol estimado','weight')
gdl_G=armar_grafo(gdl_nd,gdl_ed,'type','weight')
hgo_G=armar_grafo(hgo_nd,hgo_ed,'type','weight')
mad_G=armar_grafo(mad_nd,mad_ed,'rol estimado','weight')
mtv_G=armar_grafo(mtv_nd,mtv_ed,'rol estimado','node size')
oax_G=armar_grafo(oax_nd,oax_ed,'rol','weight')
sao_G=armar_grafo(sao_nd,sao_ed,'rol estimado','weight')
scl_G=armar_grafo(scl_nd,scl_ed,'type','weight')


In [ ]:
#-------------------------------------------------
# Attribute Mixing Matrices - Original graphs
#-------------------------------------------------
#       Obtenemos una matriz cuya entrada (i,j) es la fracción de aristas 
#       que van de un eje con rol i a un eje con rol j, en la red original


#cada rol lo asignamos a un índice del 1 al 5
rol_map_esp={'Generador de conocimiento':0,'Habilitador':1,'Promotor':2,
             'Vinculador':3,'Articulador':4,'Comunidad':5}

rol_map_ing={'Knowledge Generator':0,'Enabler':1,
             'Promoter':2,'Linker':3,'Articulator':4,'Community':5}

capitals = {'CABA': caba_G, 'CDMX':cdmx_G, 'Santiago de Chile': scl_G, 
            'Montevideo': mtv_G, 'Madrid': mad_G, 'Sao Paulo': sao_G}

#aquí vamos a almacenar las matrices
attr_mix_matrices = capitals 

#conseguimos cada una de las attribute mixing matrices
for capital_str in capitals.keys():
    if capital_str=='Montevideo':
        print('Matriz de artibuto Rol para la ciudad de '+capital_str)
        print('')
        M=nx.attribute_mixing_matrix(capitals[capital_str],'rol',mapping=rol_map_ing)
        print(M)
        print('')
        print('')
    else:
        print('Matriz de artibuto Rol para la ciudad de '+capital_str)
        print('')
        M=nx.attribute_mixing_matrix(capitals[capital_str],'rol',mapping=rol_map_esp)
        print(M)
        print('')
        print('')
    attr_mix_matrices[capital_str] = M

            
# Creamos la instancia de una figura, con subplots
fig = plt.figure(figsize = (20,20)) # ancho x alto
ax1 = fig.add_subplot(3, 3, 1) # row, column, position
ax1.set_title('Matriz de artibuto Rol para la ciudad de CABA')

ax2 = fig.add_subplot(3, 3, 2)
ax2.set_title('Matriz de artibuto Rol para la ciudad de CDMX')

ax3 = fig.add_subplot(3, 3, 3)
ax3.set_title('Matriz de artibuto Rol para la ciudad de Santiago de Chile')

ax4 = fig.add_subplot(3, 3, 4)
ax4.set_title('Matriz de artibuto Rol para la ciudad de Montevideo')

ax5 = fig.add_subplot(3, 3, 5)
ax5.set_title('Matriz de artibuto Rol para la ciudad de Madrid')

ax6 = fig.add_subplot(3, 3, 6)
ax6.set_title('Matriz de artibuto Rol para la ciudad de Sao Paulo')


# We use ax parameter to tell seaborn which subplot to use for this plot
sns.heatmap(data=attr_mix_matrices['CABA'], ax=ax1, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)
sns.heatmap(data=attr_mix_matrices['CDMX'], ax=ax2, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)
sns.heatmap(data=attr_mix_matrices['Santiago de Chile'], ax=ax3, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)
sns.heatmap(data=attr_mix_matrices['Montevideo'], ax=ax4, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)
sns.heatmap(data=attr_mix_matrices['Madrid'], ax=ax5, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)      
sns.heatmap(data=attr_mix_matrices['Sao Paulo'], ax=ax6, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)      


fig   

In [ ]:
#----------------------
# Subgrafos Nucleares 
#----------------------
#    Obtenemos los -core- nodes (nodos núcleo) de cada una de las redes capitales, 
#    donde el grado total de un nodo núcleo es al menos 3

caba_core=nx.k_core(caba_G,k=3)
cdmx_core=nx.k_core(cdmx_G,k=3)
scl_core=nx.k_core(scl_G,k=3)
mad_core=nx.k_core(mad_G,k=3)
mtv_core=nx.k_core(mtv_G,k=3)
sao_core=nx.k_core(sao_G,k=3)

#----------------------
# GraphMLs Nucleares
#----------------------
#    Ahora, convertimos cada uno de los subgrafos nucleares que obtuvimos
#    en un archivo .graphml

nx.write_graphml(caba_core,'Gephi_core/CABA core graph.graphml')
nx.write_graphml(cdmx_core,'Gephi_core/CDMX core graph.graphml')
nx.write_graphml(scl_core,'Gephi_core/Scl core graph.graphml')
nx.write_graphml(mad_core,'Gephi_core/Mad core graph.graphml')
nx.write_graphml(mtv_core,'Gephi_core/Mtv core graph.graphml')
nx.write_graphml(sao_core,'Gephi_core/Sao core graph.graphml')

#----------------------
# Subgrafos Corteza 
#----------------------
#    Obtenemos los -crust- nodes (nodos corteza) de cada una de las redes capitales, 
#    donde el grado total de un nodo corteza es menor a 3

caba_crust=nx.k_crust(caba_G,k=3)
cdmx_crust=nx.k_crust(cdmx_G,k=3)
scl_crust=nx.k_crust(scl_G,k=3)
mad_crust=nx.k_crust(mad_G,k=3)
mtv_crust=nx.k_crust(mtv_G,k=3)
sao_crust=nx.k_crust(sao_G,k=3)

#----------------------
# GraphMLs Corteza  
#----------------------
#    Ahora, convertimos cada uno de los subgrafos corteza que obtuvimos
#    en un archivo .graphml

nx.write_graphml(caba_crust,'Gephi_crust/CABA crust graph.graphml')
nx.write_graphml(cdmx_crust,'Gephi_crust/CDMX crust graph.graphml')
nx.write_graphml(scl_crust,'Gephi_crust/Scl crust graph.graphml')
nx.write_graphml(mad_crust,'Gephi_crust/Mad crust graph.graphml')
nx.write_graphml(mtv_crust,'Gephi_crust/Mtv crust graph.graphml')
nx.write_graphml(sao_crust,'Gephi_crust/Sao crust graph.graphml')

In [ ]:
#------------------------------------------------
#  Attribute Mixing Matrices - Core graphs
#------------------------------------------------
#     Para cada uno de los grafos corteza que conseguimos, 
#     Obtenemos una matriz cuya entrada (i,j) es la fracción 
#     de aristas que van de un eje con rol i a un eje con rol j


rol_map_esp={'Generador de conocimiento':0,'Habilitador':1,'Promotor':2,
             'Vinculador':3,'Articulador':4,'Comunidad':5}

rol_map_ing={'Knowledge Generator':0,'Enabler':1,'Promoter':2,
             'Linker':3,'Articulator':4,'Community':5}

capitals_cores={'CABA': caba_core, 'CDMX':cdmx_core, 'Santiago de Chile': scl_core, 
                'Montevideo': mtv_core, 'Madrid': mad_core, 'Sao Paulo': sao_core}
attr_mix_matrices = capitals_cores 
        


for capital_str in capitals.keys():
    if capital_str=='Montevideo':
        print('Matriz de artibuto Rol para el núcleo (k=3) de '+capital_str)
        print('')
        M=nx.attribute_mixing_matrix(capitals_cores[capital_str],'rol',mapping=rol_map_ing)
        print(M)
        print('')
        print('')
    else:
        print('Matriz de artibuto Rol para el núcleo (k=3) de '+capital_str)
        print('')
        M=nx.attribute_mixing_matrix(capitals_cores[capital_str],'rol',mapping=rol_map_esp)
        print(M)
        print('')
        print('')
    attr_mix_matrices[capital_str] = M

        
        
# Creamos la instancia de una figura, con subplots
fig = plt.figure(figsize = (20,20)) # width x height
ax1 = fig.add_subplot(3, 3, 1) # row, column, position
ax1.set_title('Matriz de artibuto Rol para el núcleo (k=3) de CABA')

ax2 = fig.add_subplot(3, 3, 2)
ax2.set_title('Matriz de artibuto Rol para el núcleo (k=3) de CDMX')

ax3 = fig.add_subplot(3, 3, 3)
ax3.set_title('Matriz de artibuto Rol para el núcleo (k=3) de Santiago de Chile')

ax4 = fig.add_subplot(3, 3, 4)
ax4.set_title('Matriz de artibuto Rol para el núcleo (k=3) de Montevideo')

ax5 = fig.add_subplot(3, 3, 5)
ax5.set_title('Matriz de artibuto Rol para el núcleo (k=3) de Madrid')

ax6 = fig.add_subplot(3, 3, 6)
ax6.set_title('Matriz de artibuto Rol para el núcleo (k=3) de Sao Paulo')


# We use ax parameter to tell seaborn which subplot to use for this plot
sns.heatmap(data=attr_mix_matrices['CABA'], ax=ax1, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)
sns.heatmap(data=attr_mix_matrices['CDMX'], ax=ax2, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)
sns.heatmap(data=attr_mix_matrices['Santiago de Chile'], ax=ax3, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)
sns.heatmap(data=attr_mix_matrices['Montevideo'], ax=ax4, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)
sns.heatmap(data=attr_mix_matrices['Madrid'], ax=ax5, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)      
sns.heatmap(data=attr_mix_matrices['Sao Paulo'], ax=ax6, square=True, cbar_kws={'shrink': .3}, annot=True, annot_kws={'fontsize': 12}, vmin=0.0, vmax=0.6)      


fig


In [ ]:
#----------------------------
# Medidas Globales
#----------------------------
#     Ahora obtenemos, para cada una de las capitales, una serie 
#     pandas.core.series.Series, utilizando la función mean(), 
#     con la que obtenemos las siguientes centralidades 
#           - timeset
#           - mentions    
#           - avg strength                    promedio de la fuerza de los nodos
#           - weight                          promedio del peso de los nodos
#           - indegree                        promedio del grado hacia los nodos
#           - outdegree                       promedio del grado fuera de los nodos
#           - Degree                          promedio de in+out
#           - weighted indegree               promedio del grado hacia los nodos con pesos de acuerdo a intensidades
#           - weighted outdegree             promedio del grado fuera de los nodos con pesos de acuerdo a intensidades
#           - Weighted Degree                promedio in+out de acuerdo a intensidades
#           - Eccentricity                   promedio de eccentricidad 
#           - closnesscentrality              
#           - harmonicclosnesscentrality      
#           - betweenesscentrality          
#           - modularity_class                
#           - Authority                       promedio de autoridad de los nodos de acuerdo al aloritmo HITS
#           - Hub                             promedio de Hubs de los nodos de acuerdo al aloritmo HITS
#           - componentnumber                 número de componentes
#           - strongcompnum                  número de componentes fuertemente conectados
#           - clustering                     promedio de coeficiente de clustering
#           - triangles                      
#           - eigencentrality                 promedio de centralidades eigenvectores

capitals_stats={'CABA': caba_nd, 'CDMX':cdmx_nd, 'Santiago de Chile': scl_nd, 
                'Montevideo': mtv_nd, 'Madrid': mad_nd, 'Sao Paulo': sao_nd}


averages={city: stats.mean() for city,stats in capitals_stats.items()}
#averages['Sao Paulo']

In [ ]:
#Ejemplo: Aquí vamos a conseguir el CSV para el núcleo de CDMX, en donde conocemos algunas de sus métricas

city='CDMX'
avg=averages[city]
datafr=avg.copy().to_frame().T
datafr.drop(['timeset'], axis=1, inplace=True)
if 'type' in datafr.columns:
    datafr.drop(['type'], axis=1, inplace=True)

all_columns = datafr.columns.values.tolist()
print(all_columns)
print('')
print('')
print('')
datafr


In [ ]:
countries={'Montevideo':'Uruguay', 'CABA':'Argentina', 'CDMX':'México',
          'Madrid':'España', 'Sao Paulo': 'Brasil', 'Santiago de Chile': 'Chile'}


list_concat=[]
for city,avg in averages.items():
    datafr=avg.copy().to_frame().T
    
    #borramos del set de columnas a aquellos atributos que no son significantes 
    datafr.drop(['timeset', 'componentnumber'], axis=1, inplace=True)
    if 'type' in datafr.columns:
        datafr.drop(['type'], axis=1, inplace=True)
    
    #añadimos datafr a la lista de dataframes que vamos a concatenar
    list_concat.append(datafr)
    
    #añadimos el atributo que corresponde a el nombre de ciudad y país
    datafr.insert(0, 'País', [countries[city]], True) 
    datafr.insert(0, 'Ciudad', [city], True) 

df_concat=pd.concat(list_concat, ignore_index=True)
df_concat

In [ ]:
df_concat.isnull().sum()

In [ ]:
#Quitar promedios que en la pagina no mencionan que describen una propiedad global del grafo

df_concat=df_concat.drop('closnesscentrality',axis=1)
df_concat=df_concat.drop('harmonicclosnesscentrality',axis=1)
df_concat=df_concat.drop('modularity_class',axis=1)
df_concat=df_concat.drop('triangles',axis=1)
df_concat=df_concat.drop('eigencentrality',axis=1)
df_concat=df_concat.drop('pageranks',axis=1)
df_concat=df_concat.drop('ego',axis=1)
df_concat=df_concat.drop('betweenesscentrality',axis=1)
df_concat=df_concat.drop('Hub',axis=1)


In [ ]:
df_concat['weight'][3]=df_concat['node size'][3]
df_concat=df_concat.drop('node size',axis=1)
df_concat=df_concat.drop('strongcompnum',axis=1)
df_concat=df_concat.drop('Authority',axis=1)

In [ ]:
df_concat

In [ ]:
"""
Global characteristics we may compute:

*Diameter  
*Radius
**Average path length
**Transitivity
*Global Efficiency
***Modularity
*Assortativity Coefficient
Small Worldness


*   = computed already, not defined for direted graph
**  = computed for undirected graph, also defined for directed graph
*** = computed manually
"""




In [ ]:
#--------------
# Diameter
#--------------
capitals={'CABA': caba_G, 'CDMX':cdmx_G, 'Santiago de Chile': scl_G, 
          'Montevideo': mtv_G, 'Madrid': mad_G, 'Sao Paulo': sao_G}


print('-------------------------------------------')
print('Diámetro en Cores dirigidos')
print('-------------------------------------------')
for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.diameter(graph)))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('')

print('-------------------------------------------')
print('Diámetro en Cores no dirigidos')
print('-------------------------------------------')
for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.diameter(nx.to_undirected(graph))))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('')
        

In [ ]:
#--------------
# Radius
#--------------

print('-------------------------------------------')
print('Radio en Cores dirigidos')
print('-------------------------------------------')
for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.radius(graph)))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('')

print('-------------------------------------------')
print('Radio en Cores no dirigidos')
print('-------------------------------------------')
for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.radius(nx.to_undirected(graph))))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('') 

In [ ]:
#---------------------------------
# Camino más corto promedio
#---------------------------------

# parece que aunque las graficas dirigidas no sean conexas, el camino 
# más corto promedio si se está pudiendo computar, y nos da un resultado.. menor?

print('-------------------------------------------')
print('Camino más corto promedio en Cores dirigidos')
print('-------------------------------------------')
for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.average_shortest_path_length(graph)))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('')

print('-------------------------------------------')
print('Camino más corto promedio en Cores no dirigidos')
print('-------------------------------------------')
for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.average_shortest_path_length(nx.to_undirected(graph))))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('')  

In [ ]:
#---------------------------------
# Transitividad
#---------------------------------

# parece que aunque las graficas dirigidas no sean conexas, el camino 
# más corto promedio si se está pudiendo computar, y nos da un resultado.. menor?
print('----------------------------------')
print('Transitividad en Cores dirigidos')
print('----------------------------------')

for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.transitivity(graph)))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('')

print('----------------------------------')
print('Transitividad en Cores no dirigidos')
print('----------------------------------')

for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.transitivity(nx.to_undirected(graph))))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('') 

In [ ]:
#-----------------------
# Eficiencia Global
#-----------------------
# Solo esta definida para las versiones no dirigidas de nuestras redes

print('----------------------------------')
print('Eficiencia Global en Cores dirigidos')
print('----------------------------------')

for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.global_efficiency(graph)))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('')

print('----------------------------------')
print('Eficiencia Global en Cores no dirigidos')
print('----------------------------------')

for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.global_efficiency(nx.to_undirected(graph))))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('') 

In [ ]:
#-------------------
# Modularidad
#-------------------

#modularidad Parece no estar definido en este esquema, pero podemos obtener esta medida de Gephi

print('----------------------------------')
print('Modularidad en Cores dirigidos')
print('----------------------------------')

for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.community.modularity(graph)))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('')

print('----------------------------------')
print('Modularidad en Cores no dirigidos')
print('----------------------------------')

for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.community.modularity(nx.to_undirected(graph))))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('') 

In [ ]:
#---------------------------
# Assortativity coefficient
#---------------------------

print('----------------------------------')
print('Coef. Asortatividad en Cores dirigidos')
print('----------------------------------')

for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.degree_assortativity_coefficient(graph)))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('')

print('----------------------------------')
print('Coef. Asortatividad en Cores no dirigidos')
print('----------------------------------')

for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.degree_assortativity_coefficient(nx.to_undirected(graph))))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('') 

d1 = oax_G.to_undirected().degree('IODEMC')
d2 = scl_G.to_undirected().degree('CORFO')
print(d1, d2)

In [ ]:
#------------------
# Small Worldness
#------------------

#networkx.algorithms.smallworld.random_reference
print('----------------------------------')
print('Small Worldness en Cores dirigidos')
print('----------------------------------')

for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.algorithms.smallworld.sigma(graph,niter=1,nrand=2)))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('')

print('----------------------------------')
print('Small Worldness en Cores no dirigidos')
print('----------------------------------')

for city, graph in capitals.items():
    try:
        print(city+str(': ')+str(nx.algorithms.smallworld.sigma(graph.to_undirected(),niter=1,nrand=2)))
        print('')
    except:
        print(city+str(': ')+str(float('inf')))
        print('') 

In [ ]:
#-----------------
#  Modularidades
#-----------------
#     Estas medidas fueron tomadas no por medio de este programa, sino manualmente desde el programa Gephi
#     Metodología:
#        Tomamos tres medidas en cada red, con cada una de las resoluciones 0.8, 1.0, 1.2, con o sin ejes
#
#        Network       (CABA, CDMX, MAD, MTV SAO, SCL)
#        Resolution    (0.8, 1.0, 1.2)  
#        Edge weights  (Y/N)
#        
#        Y luego de ello tomamos la mediana de las tres medidas de resolución, a modo de tomar en cuenta variabilidad




df_concat['Modularidad con pesos (0.8)'] = [0.402, 0.484, 0.530, 0.472, 0.611, 0.686]
df_concat['Modularidad con pesos (1.0)'] = [0.524, 0.628, 0.530, 0.611, 0.610, 0.682]
df_concat['Modularidad con pesos (1.2)'] = [0.654, 0.776, 0.524, 0.756, 0.612, 0.673]


df_concat['Modularidad sin pesos (0.8)'] = [0.497, 0.602, 0.496, 0.353, 0.568, 0.673]
df_concat['Modularidad sin pesos (1.0)'] = [0.493, 0.604, 0.500, 0.348, 0.572, 0.664]
df_concat['Modularidad sin pesos (1.2)'] = [0.499, 0.601, 0.505, 0.337, 0.563, 0.661]

df_concat

In [ ]:
capitals = {'CABA': caba_G, 'CDMX':cdmx_G, 'Santiago de Chile': scl_G, 'Montevideo': mtv_G, 'Madrid': mad_G, 'Sao Paulo': sao_G}

#las columnas que vamos a aregar al DataFrame
diameter = [0]*len(capitals)
radius = [0]*len(capitals)
avg_shortest_path_length = [0]*len(capitals)
transitivity = [0]*len(capitals)
global_efficiency = [0]*len(capitals)
modularity = [0]*len(capitals)

small_worldness = [0]*len(capitals)


for city, graph in capitals.items():
    city_index=df_concat.index[df_concat['Ciudad']==city].tolist()[0]
    
    #conseguimos cada parámetro para esta ciudad
    undirected=nx.to_undirected(graph)
    diameter[city_index] =                     nx.diameter(undirected)
    radius[city_index] =                       nx.radius(undirected)
    avg_shortest_path_length[city_index] =     nx.average_shortest_path_length(undirected)
    transitivity[city_index] =                 nx.transitivity(undirected)
    global_efficiency[city_index] =            nx.global_efficiency(undirected)
    small_worldness[city_index] =              nx.nx.algorithms.smallworld.sigma(graph.to_undirected(),niter=1,nrand=5)

    
    #espacio para modularidad
    
    #deg_assortativity_coefficient[city_index]= nx.degree_assortativity_coefficient(undirected)
    
df_concat['Diámetro'] = diameter
df_concat['Radio'] = radius
df_concat['Camino más corto promedio'] = avg_shortest_path_length
df_concat['Transitividad'] = transitivity
df_concat['Eficiencia Global'] = global_efficiency
#df_concat['Coeficiente de Asortatividad de Grado'] = deg_assortativity_coefficient
df_concat['Small Worldness']= small_worldness
df_concat


In [ ]:
#Cambiar los nombres de los atributos del DataFrame
#algunos nombres de columnas no fueron modificados para facilitar legibilidad
df_concat.rename(columns={'mentions': 'Menciones', 'Eccentricity': 'Eccentricidad', 'clustering':'Clustering'}, inplace=True)
df_concat

In [ ]:
df_concat.to_csv('Tidy_DataFrame.csv',index=False)

### Explorando el caso de Oaxaca y Santiago de Chile

Una de las observaciones que hizo el equipo de GED fue acerca de las redes de Oaxaca y Santiago de Chile. En Oaxaca, el nodo más prominente es IODEMC, mientras que en SCL el nodo más prominente es CORFO. Ambos nodos presentan mayor número de menciones, mayor número de flechas dirigidas hacia dichos nodos, y mayor grado. Por ende, estos nodos también resultan prominentes en casi todas las métricas nodales. 

Sin embargo, al remover IODEMC de Oaxaca, la red se desestabiliza:
- aloha 

Por el otro lado, al remover CORFO de SCL, la red no logra desestabilizarse tanto como la red previa. 




In [ ]:
from statistics import mean 
from statistics import stdev 

In [ ]:
#------------------------
# Triangles
#------------------------

#obtaining IODEMC's triangle number in OAX's network
print(nx.triangles(oax_G.to_undirected(), 'IODEMC'))

#obtaining CORFO's triangle number in SCL's network
print(nx.triangles(scl_G.to_undirected(), 'CORFO'))

In [ ]:
#------------------------
# Path length
#------------------------
#average shortest path length from any other node to our nodes of interest.

#obtaining IODEMC's path length in OAX's network
print( mean(nx.shortest_path_length(oax_G, 'IODEMC').values()) )

#obtaining CORFO's path length in SCL's network
print( mean(nx.shortest_path_length(scl_G, 'CORFO').values()) )

In [ ]:
#------------------------
# Global efficiency
#------------------------
#average shortest path length from any other node to our nodes of interest.

#obtaining IODEMC's global efficiency in OAX's network
shortest_path_lengths_oax = nx.shortest_path_length(oax_G, 'IODEMC')
del (shortest_path_lengths_oax['IODEMC'])
print( mean(shortest_path_lengths_oax.values()) ) 

#obtaining CORFO's global efficiency in SCL's network
shortest_path_lengths_scl = nx.shortest_path_length(scl_G, 'CORFO')
del (shortest_path_lengths_scl['CORFO'])
print( mean(shortest_path_lengths_scl.values()) ) 

In [ ]:
#------------------------
# Local efficiency
#------------------------
#average shortest path length from any neighbor node to our nodes of interest.

def nodes_connected(G, u, v):
    return u in G.neighbors(v)


#obtaining IODEMC's global efficiency in OAX's network
shortest_paths_ngb_oax = {key:value for key,value in shortest_path_lengths_oax.items() if nodes_connected(oax_G, 'IODEMC', key)}
print( mean([1/x for x in shortest_paths_ngb_oax.values()]) ) 

#obtaining CORFO's global efficiency in SCL's network
shortest_paths_ngb_scl = {key:value for key,value in shortest_path_lengths_scl.items() if nodes_connected(scl_G, 'CORFO', key)}
print( mean([1/x for x in shortest_paths_ngb_scl.values()]) ) 

In [ ]:
#------------------------
# Within module z-score
#------------------------
#   Degree to which a node is connected to other nodes inside the same community
#   Helper functions:

def nodes_connected(G, u, v):
    return u in G.neighbors(v)

def deg_in_module(G, M, v):
    #degree of node inside its community
    k=0                                                      
    for x in M:
        if nodes_connected(G, v, x) or nodes_connected(G, x, v): k+=1
    return k


#    Node:       IODEMC
#    Network:    oax_G
#    Community:  Articulators

articulators = [x for x,y in oax_G.nodes(data=True) if y['rol']=='Articulador']
ki = deg_in_module(oax_G, articulators, 'IODEMC')                                 #deg in module of IODEMC
in_mod_degrees = [deg_in_module(oax_G, articulators, v) for v in articulators ]
ksi = mean(in_mod_degrees)                                                        #average degree 
sigmasi = stdev(in_mod_degrees)                                                   #standard deviation of degrees


print('K_i = '+str(ki) )
print('K_Si = '+str(ksi) )
print('Sigma_Si = '+str(sigmasi) )
print('IODEMCs within module z-score is: '+ str((ki-ksi)/sigmasi) )
print('-----------------------------------------------')



#    Node:       CORFO
#    Network:    oax_G
#    Community:  Articulators

articulators = [x for x,y in scl_G.nodes(data=True) if y['rol']=='Articulador']
ki = deg_in_module(scl_G, articulators, 'CORFO')                                 #deg in module of IODEMC
in_mod_degrees = [deg_in_module(scl_G, articulators, v) for v in articulators ]
ksi = mean(in_mod_degrees)                                                        #average degree 
sigmasi = stdev(in_mod_degrees)                                                   #standard deviation of degrees


print('K_i = '+str(ki) )
print('K_Si = '+str(ksi) )
print('Sigma_Si = '+str(sigmasi) )
print('CORFOs within module z-score is: '+ str((ki-ksi)/sigmasi) )
print('-----------------------------------------------')




In [ ]:
#----------------------------
# Participation coefficient
#----------------------------
#   Relation between edges connecting node with other communities, and total number of edges from node
#   Helper functions:

def nodes_connected(G, u, v):
    return u in G.neighbors(v)

def deg_in_module(G, M, v):
    #degree of node inside its community
    k=0                                                      
    for x in M:
        if nodes_connected(G, v, x) or nodes_connected(G, x, v): k+=1
    return k


#    Node:       IODEMC
#    Network:    oax_G
#    Community:  Articulators

roles=['Articulador', 'Habilitador', 'Generador de conocimiento', 'Vinculador', 'Comunidad', 'Promotor']
communities = {rol: [x for x,y in oax_G.nodes(data=True) if y['rol']==rol] for rol in roles}
d = oax_G.degree('IODEMC')
print(d)
p=1
for rol in roles:
    p -= (deg_in_module(oax_G, communities[rol], 'IODEMC'))**2 / d**2    #we want to check degree of 'IODEMC' 

print('IODEMCs participation coefficient is: '+ str(p) )
print('-----------------------------------------------')



#    Node:       CORFO
#    Network:    oax_G
#    Community:  Articulators

roles=['Articulador', 'Habilitador', 'Generador de conocimiento', 'Vinculador', 'Comunidad', 'Promotor']
communities = {rol: [x for x,y in scl_G.nodes(data=True) if y['rol']==rol] for rol in roles}
d = scl_G.degree('CORFO')
print(d)
p=1
for rol in roles:
    p -= (deg_in_module(scl_G, communities[rol], 'CORFO'))**2 / d**2    #we want to check degree of 'IODEMC' 

print('CORFOs participation coefficient is: '+ str(p) )
print('-----------------------------------------------')